<h1>IBM Data Science Final Project</h1>

<h2>Introduction: Business Problem</h2>

<b>One of my friends , Robinson, is relocating to India and is interested in setting up a Goan Restaurant in Chennai.He has been a sea food lover since his childhood. He is looking at locations where presence of Resturants is minimal and in particular there are no Goan Cuisine Restaurants. He has asked me to leverage my data science skills and come up with ideal locations for his new venture i.e. Goan Restaurant.</b>

<h3>Data</h3>

<b> Based on my friend's requirements, below are the factors that needs to be considered</b>
1. Number of existing restaurants in each neighburhood
2. Number of Goan restaurants in each neighborhood
3. Proximity and distance

<b>To achieve the above:</b>
1. First we need to get the list of neighborhoods in Chennai city
https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai
2. We need to scrape the wikipedia page containing the neighborhoods details ans fetch them
3. Then for each neighborhood, get latitude and longitude using Google Maps API
4. Then, using the four square API,get the restaurants and Goan Restaurant in each neighborhood
5. We will then use k-means clustering method to group the neighborhoods based on how similar they are.
6. The tools for EDA will be pandas, folium, google API, matplotlib.


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<b>Scrape data from wikipedia page to get the neighborhoods</b>

In [9]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai").text

In [3]:
from bs4 import BeautifulSoup

<b>Using lxml Parser, parse the contents of the data using BeautifulSoup library</b>

In [4]:
soup=BeautifulSoup(data,'lxml')

<b>Traverse through all the list elements under the class mw_category and add all the neighborhoods to a list</b>

In [5]:
neighborhood=[]
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhood.append(row.text)

<b>Create a function that returns the latitude and longitude of a given address</b>

In [10]:
def get_lat_long(neighborhood):
    geolocator = Nominatim(user_agent="manoj483@gmail.com")
    latitude=None
    longitude=None
    location = geolocator.geocode(neighborhood,timeout=25)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    return latitude, longitude


<b>For each neighborhood in the list, get the latitude and longitude and add to a dataframe. If the latitude or longitude values are NaN, delete them.</b>

In [11]:
address_geocode_list=[]
for address in neighborhood:
    latitude, longitude = get_lat_long(address)
    address_geocode_list.append([address,latitude,longitude])
#address_geocode_list
df_geo_list = pd.DataFrame(address_geocode_list,columns=['Neighborhood','Latitude','Longitude'])
df_geo_list = df_geo_list.dropna()
df_geo_list.head()
    

,Neighborhood,Latitude,Longitude
0,Alandur,12.994373,80.194284
1,Anna Nagar,11.170349,77.351114
2,"Ashok Nagar, Chennai",13.040073,80.215925
3,Assisi Nagar,13.164610,80.233000
4,Ayanavaram,13.094616,80.235410


<b>Get latitude and longitude of Chennai</b>

In [13]:
latitude, longitude = get_lat_long('Chennai')
print ('Latitude and Longitude of Chennai is {},{}'.format(latitude,longitude))

Latitude and Longitude of Chennai is 13.0801721,80.2838331


<b>Create a map of Chennai with neighborhoods superimposed on it</b>

In [19]:
chennai_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for latitude, longitude, neighborhood in zip(df_geo_list['Latitude'], df_geo_list['Longitude'], df_geo_list['Neighborhood']):
    name = '{}'.format(neighborhood)
    name = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=name,
        color='Green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(chennai_map)  
    
chennai_map